In [6]:
import pandas as pd
import numpy as np
import datetime

In [1]:
# variables
building = "CARPARK1"
month = "June"
year = "2022"
reportName = "HISTORY TRANSACTION"
fileFormat = "csv"

In [7]:
path = "batch 2/outputFiles/entryExitMatched/CARPARK1entryExitMatched.csv"
entry_df = pd.read_csv(path)
entry_df["ENTRYDATETIME"] = pd.to_datetime(entry_df["ENTRYDATETIME"], dayfirst=True)
entry_df["EXITDATETIME"] = pd.to_datetime(entry_df["EXITDATETIME"], dayfirst=True)

In [9]:
option = "Hours"
multiple = "1"
frequency = {"Seconds": "S",
            "Minutes": "min",
            "Hours": "H"}
columns = pd.date_range(start="00:00:00", end="23:59:59", freq=multiple+frequency[option])

In [10]:
# round down datetime
def roundTime(start, end, option):
    if option == "Hours":
        return start.replace(microsecond=0, second=0, minute=0), end.replace(microsecond=0, second=0, minute=0)
    elif option == "Minutes":
        return start.replace(microsecond=0, second=0), end.replace(microsecond=0, second=0)
    elif option =="Seconds":
        return start.replace(microsecond=0), end.replace(microsecond=0)
    else:
        print("Invalid Option!")

timeRanges = []
for index, row in entry_df.iterrows():
    try:
        startTime, endTime = roundTime(row["ENTRYDATETIME"], row["EXITDATETIME"], option)
        timeRanges.append(pd.date_range(startTime, endTime, freq=multiple+frequency[option]))
    except ValueError:
        timeRanges.append(np.NaN)
entry_df["timeRange"] =  timeRanges

In [11]:
for time in columns:
    specific_time = []
    for index, row in entry_df.iterrows():
        time_found = False
        timeRanges = row["timeRange"]
        if type(timeRanges) != float:
            for timestamp in timeRanges:
                if time.time() == timestamp.time():
                    specific_time.append(1)
                    time_found = True
                    break
            if time_found == False:
                specific_time.append(0)
        else:
            specific_time.append(0)
    entry_df[time.time()] = specific_time

In [16]:
def getDate(dtObject):
    if type(dtObject) != float:
        return str(dtObject).split()[0]

entry_df["DATE"] = entry_df["ENTRYDATETIME"].apply(getDate)
byDay_df = entry_df.groupby("DATE").sum()
byDay_df.drop(columns=["S/NO.", "VEHICLE"], inplace=True)
byDay_df.reset_index(inplace=True)

In [92]:
def getWeekday(date):
    dateComponents = date.split('-')
    dtObject = datetime.date(year=int(dateComponents[0]), month=int(dateComponents[1]), day=int(dateComponents[2]))
    return dtObject.weekday()

byDay_df["DAY"] = byDay_df["DATE"].apply(getWeekday)


In [94]:
byWeekday_df = byDay_df.groupby("DAY").mean().reset_index().sort_values("DAY")

def convertToDaysOfTheWeek(num):
    weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    return weekdays[num]

byWeekday_df["DAY"] = byWeekday_df["DAY"].apply(convertToDaysOfTheWeek)

In [95]:
# create excel
batchNo = "2"
building = "SOL"
excelName = building + "heatmap" 

path = "batch " + batchNo + "/outputFiles/heatmaps/" + excelName + ".xlsx"
writer = pd.ExcelWriter(path, engine='xlsxwriter')

# add data to excel sheet
byDay_df.to_excel(writer, sheet_name="byDay", index=False)

# Get sheet for conditional formatting 
worksheet = writer.sheets['byDay']

# Add conditional formatting
rowCount = str(byDay_df.shape[0]+1)
worksheet.conditional_format('B2:Y'+rowCount, {'type': '3_color_scale',
                                                'min_type': 'min',
                                                'mid_type': 'percent',
                                                'mid_value': 25,
                                                'max_type': 'max',
                                                'min_color': '#63BE7B',
                                                'mid_color': '#FFEB84',
                                                'max_color': '#F8696B',
                                                })

# add data to excel sheet
byWeekday_df.to_excel(writer, sheet_name="byWeekday", index=False)

# Get sheet for conditional formatting 
worksheet = writer.sheets['byWeekday']

# Add conditional formatting
rowCount = str(byWeekday_df.shape[0]+1)
worksheet.conditional_format('B2:Y'+rowCount, {'type': '3_color_scale',
                                                'min_type': 'min',
                                                'mid_type': 'percent',
                                                'mid_value': 25,
                                                'max_type': 'max',
                                                'min_color': '#63BE7B',
                                                'mid_color': '#FFEB84',
                                                'max_color': '#F8696B',
                                                })

writer.save()